## Курсовая работа.

### Тема: создание чат-бота в telegram.

#### Загрузка библиотек:

In [1]:
#pip install dialogflow gensim==3.6.0 --user

In [2]:
import gensim
gensim.__version__

c:\users\rfk03\appdata\local\programs\python\python39\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


'3.6.0'

In [3]:
!python --version

Python 3.9.0


In [4]:
#pip install python-telegram-bot==13.3

In [5]:
import telegram
telegram.__version__

'13.3'

In [6]:
#pip install annoy --use-pep517

In [7]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [8]:
#from tqdm.notebook import tqdm

In [9]:
#from tqdm import *

In [10]:
#pip install ipywidgets --user

#### Подготовка и препроцессинг данных для вопросно-ответной системы:

In [11]:
#assert False
#Small preprocess of the answers
#Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`

question = None
written = False

with open("prepared_answers.txt", "w", encoding="utf8") as fout:  
    with open("Otvety.txt", "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\4193150760.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [12]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [13]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 50000:
            break

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\1981494656.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

#### Создание и сохранение модели вопросно-ответной системы:

In [14]:
# Классифицируем текст с помощью FastText

sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

c:\users\rfk03\appdata\local\programs\python\python39\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [15]:
# Создаем Индексы для вопросов-ответов
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)


modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

        if counter > 50000:
            break

ft_index.build(10)
ft_index.save('speaker.ann')

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\4283737800.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\4283737800.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\4283737800.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word]


True

In [16]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [17]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[1383, 1947]

#### Загрузка и подготовка данных для продуктовой модели:

In [19]:
shop_data = pd.read_csv("kaspi_reviews.csv")
#shop_data = shop_data.iloc[:5000, :]
shop_data.head()

,Unnamed: 0,text,plus,minus,language,rating,category
0,0,"Парфюм оригинальный. Всё, как в профессиональн...",NaN,NaN,russian,5.0,perfumes
1,1,Мой любимый DKNY. Запах просто шикарный. Спаси...,NaN,NaN,russian,5.0,perfumes
2,2,NaN,Шикарный запах! Безумно нравится! Не пожалеете...,NaN,russian,5.0,perfumes
3,3,NaN,Иісі қатты ұнады.,NaN,kazakh,5.0,perfumes
4,4,"Запах просто классный, как раз то, что мне нуж...","Мне все понравилось - и качество товара, и зап...","Единственное, что мне не понравилось, это то, ...",russian,4.0,perfumes


In [20]:
shop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119048 entries, 0 to 119047
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  119048 non-null  int64  
 1   text        59479 non-null   object 
 2   plus        83900 non-null   object 
 3   minus       36879 non-null   object 
 4   language    119048 non-null  object 
 5   rating      119048 non-null  float64
 6   category    119048 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 6.4+ MB


In [21]:
shop_data['category'].value_counts()

category
smartphones          45605
big-home-appl        12226
small-home-appl      12093
kitchen-home-appl    11301
headphones            6294
beauty                6260
tires                 5538
climate-equipment     4960
car-electronics       3246
perfumes              2809
watches               2276
car-audio             2065
wearables             1938
power-banks            753
portable-speakers      597
books                  563
memory-cards           524
Name: count, dtype: int64

In [22]:
shop_data = shop_data.loc[shop_data['language'] == 'russian']

In [23]:
shop_data = shop_data.drop(['Unnamed: 0'], axis=1)
shop_data = shop_data.drop(['language'], axis=1)

In [24]:
shop_data['text'] = shop_data['text'].fillna('Нет описания')
shop_data['plus'] = shop_data['plus'].fillna('Плюсы отсутствуют')
shop_data['minus'] = shop_data['minus'].fillna('Минусы отсутствуют')

In [25]:
shop_data['category'] = shop_data['category'].replace({'smartphones': 'смартфоны'})
shop_data['category'] = shop_data['category'].replace({'big-home-appl': 'большая бытовая техника'})
shop_data['category'] = shop_data['category'].replace({'small-home-appl': 'малая бытовая техника'})
shop_data['category'] = shop_data['category'].replace({'kitchen-home-appl': 'бытовая техника для кухни'})
shop_data['category'] = shop_data['category'].replace({'headphones': 'наушники'})
shop_data['category'] = shop_data['category'].replace({'beauty': 'косметика'})
shop_data['category'] = shop_data['category'].replace({'tires': 'шины'})
shop_data['category'] = shop_data['category'].replace({'climate-equipment': 'климатическое оборудование'})
shop_data['category'] = shop_data['category'].replace({'car-electronics': 'автомобильная электроника'})
shop_data['category'] = shop_data['category'].replace({'perfumes': 'парфюмерия'})
shop_data['category'] = shop_data['category'].replace({'watches': 'часы'})
shop_data['category'] = shop_data['category'].replace({'car-audio': 'аудиосистемы для авто'})
shop_data['category'] = shop_data['category'].replace({'wearables': 'аксессуары'})
shop_data['category'] = shop_data['category'].replace({'power-banks': 'павербанки'})
shop_data['category'] = shop_data['category'].replace({'portable-speakers': 'портативные наушники'})
shop_data['category'] = shop_data['category'].replace({'books': 'книги'})
shop_data['category'] = shop_data['category'].replace({'memory-cards': 'карты памяти'})

In [26]:
shop_data['text_summ'] = shop_data['text'] + " " + shop_data["plus"]+ " " + shop_data["minus"]
shop_data['text_prep'] = shop_data['text_summ'].apply(lambda x: preprocess_txt(str(x)))

In [27]:
shop_data.reset_index(drop=True, inplace=True)

In [28]:
shop_data.head(5)

,text,plus,minus,rating,category,text_summ,text_prep
0,"Парфюм оригинальный. Всё, как в профессиональн...",Плюсы отсутствуют,Минусы отсутствуют,5.0,парфюмерия,"Парфюм оригинальный. Всё, как в профессиональн...","[парфюм, оригинальный, профессиональный, магаз..."
1,Мой любимый DKNY. Запах просто шикарный. Спаси...,Плюсы отсутствуют,Минусы отсутствуют,5.0,парфюмерия,Мой любимый DKNY. Запах просто шикарный. Спаси...,"[любимый, dkny, запах, шикарный, kaspikz, parf..."
2,Нет описания,Шикарный запах! Безумно нравится! Не пожалеете...,Минусы отсутствуют,5.0,парфюмерия,Нет описания Шикарный запах! Безумно нравится!...,"[описание, шикарный, запах, безумно, нравиться..."
3,"Запах просто классный, как раз то, что мне нуж...","Мне все понравилось - и качество товара, и зап...","Единственное, что мне не понравилось, это то, ...",4.0,парфюмерия,"Запах просто классный, как раз то, что мне нуж...","[запах, классный, летний, принимать, душа, бры..."
4,"Парфюм хороший, оригинал! Магазин Parfum-Shop-...",Плюсы отсутствуют,Минусы отсутствуют,5.0,парфюмерия,"Парфюм хороший, оригинал! Магазин Parfum-Shop-...","[парфюм, хороший, оригинал, магазин, parfumsho..."


In [29]:
shop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112049 entries, 0 to 112048
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   text       112049 non-null  object 
 1   plus       112049 non-null  object 
 2   minus      112049 non-null  object 
 3   rating     112049 non-null  float64
 4   category   112049 non-null  object 
 5   text_summ  112049 non-null  object 
 6   text_prep  112049 non-null  object 
dtypes: float64(1), object(6)
memory usage: 6.0+ MB


#### Подготовка и разбиение данных для обучения продуктовой модели:

In [30]:
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [31]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text_prep'].values]

In [32]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels, random_state = 42)

#### Обучение продуктовой модели:

In [34]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9939057464662902

In [36]:
assert True
import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [38]:
np.mean(tfidf_vect.idf_)

11.680515294785451

In [39]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'описание': 9.74322058047026,
 'выбирать': 7.097045783086137,
 'остановиться': 10.436367761030205,
 'превзойти': 12.045805673464306,
 'ожидание': 12.045805673464306,
 'блендер': 12.045805673464306,
 'супер': 12.045805673464306,
 'цена': 12.045805673464306,
 'качество': 12.045805673464306,
 'устроить': 12.045805673464306,
 'советовать': 12.045805673464306,
 'довольный': 12.045805673464306,
 'каспи': 12.045805673464306,
 'крутой': 12.045805673464306,
 'возможность': 12.045805673464306,
 'приобретать': 12.045805673464306,
 'чтолибо': 12.045805673464306,
 'минус': 12.045805673464306,
 'отсутствовать': 12.045805673464306,
 'httptfileruforumssearchphpqтургорampf0ampgampql0ampaampdampio1ampoampcp1ampsgt0ampslt0': 12.045805673464306,
 'httpicqmobileinfo': 12.045805673464306,
 'новый': 12.045805673464306,
 'рабочий': 12.045805673464306,
 'версия': 11.64034056535614,
 'хороший': 12.045805673464306,
 'стиральный': 12.045805673464306,
 'машина': 12.045805673464306,
 'доставка': 12.045805673464306

In [40]:
# Создаем Индексы для продуктовых данных

ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "category"],shop_data.loc[i, "text_summ"], shop_data.loc[i, "rating"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text_prep"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\454435049.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/112049 [00:00<?, ?it/s]

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\454435049.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\454435049.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)


True

In [41]:
assert True
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [42]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [43]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([2586], [1.4142135381698608])

#### Создание взаимодействия с telegram:

In [44]:
#pip install python-telegram-bot --upgrade

In [ ]:
updater = Updater("6262049149:AAEHi2V7bLxDj2i851WKobSiCk7bylGVdfY", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Здравствуйте. Что Вы хотите узнать?')
def textMessage3(update, context):
    response = 'Сообщение получено: ' + str(update.message.text)
    context.bot.send_message(chat_id=update.message.chat_id, text=response)
    
    
    
def startCommand2(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Я не понимаю Вас! :(")
        return
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\770476893.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\770476893.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\users\rfk03\appdata\local\programs\python\python39\lib\site-packages\telegram\ext\dispatcher.py", line 442, in process_update
    handler.handle_update(update, self, check, context)
  File "c:\users\rfk03\appdata\local\programs\python\python39\lib\site-packages\telegram\ext\handler.py", line 160, in handle_update
    return self.callback(update, context)
  File "C:\Users\rfk03\AppData\Local\Temp\ipykernel_13700\2599502301.py", line 25, in text